In [ ]:
import explore.exploring_functions
%cd ..

# Tutorial: Block-wise missing data generation

## Prerequisites

We will need the following libraries installed: matplotlib

## Step 1: Import required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from imml.ampute import Amputer
from imml.utils import DatasetUtils
from imml.impute import get_observed_mod_indicator

In [ ]:
from tueplots import axes, bundles
plt.rcParams.update(**bundles.icml2022(), **axes.lines())
for key in ["axes.labelsize", "axes.titlesize", "font.size", "legend.fontsize", "xtick.labelsize", "ytick.labelsize"]:
    plt.rcParams[key] += 6
# plt.rcParams.update({'text.usetex': False, "svg.fonttype": 'none'})

## Step 2: Load the dataset

Let's create a random multi-view dataset with 1000 samples and 5 views.

In [ ]:
RANDOM_STATE = 7
n_views = 4
n_samples = 10
Xs = [pd.DataFrame(np.random.default_rng(RANDOM_STATE).random((n_samples, 10))) for i in range(n_views)]

## Step 3: Apply missing data mechanism (Amputation)

Using Amputer, we randomly introduce missing data to simulate a scenario where some modalities are missing. Here, 30% of the samples will be incomplete.

In [ ]:
mechanism = "mem"
transformed_Xs = Amputer(mechanism=mechanism, p=0.8, random_state=RANDOM_STATE).fit_transform(Xs)

We can visualize which modalities are missing using a binary color map, where black means observed and white means empty (missing).

In [ ]:
xlabel,ylabel = "Modality", "Samples"
observed_view_indicator = get_observed_mod_indicator(transformed_Xs).sort_values(list(range(len(transformed_Xs))))
observed_view_indicator.columns = observed_view_indicator.columns + 1
plt.pcolor(observed_view_indicator, cmap="binary", edgecolors="black", vmin=0., vmax=2.)
plt.xticks(np.arange(0.5, len(observed_view_indicator.columns), 1), observed_view_indicator.columns)
plt.yticks(np.arange(1.5, len(observed_view_indicator), 2), np.arange(2, len(observed_view_indicator)+2, 2))
_ = plt.xlabel(xlabel), plt.ylabel(ylabel)

In [ ]:
_,ax = plt.subplots(1,1, figsize= (3.18,2.2))
# plt.figure(figsize= (3,2.5))
transformed_Xs = Amputer(mechanism=mechanism, p=0., random_state=RANDOM_STATE).fit_transform(Xs)
observed_view_indicator = get_observed_mod_indicator(transformed_Xs).sort_values(list(range(len(transformed_Xs))))
observed_view_indicator.columns = observed_view_indicator.columns + 1
ax.pcolor(observed_view_indicator, cmap="binary", edgecolors="black", vmin=0., vmax=2.)
ax.set_title("Fully observed")
ax.set_xticks(np.arange(0.5, len(observed_view_indicator.columns), 1), observed_view_indicator.columns)
ax.set_yticks(np.arange(1.5, len(observed_view_indicator), 2), np.arange(2, len(observed_view_indicator)+2, 2))
ax.set_xlabel(xlabel), ax.set_ylabel(ylabel)

plt.savefig("paper_figures/amputation_fully.pdf")
plt.savefig("paper_figures/amputation_fully.svg")

## Step 4: Visualize different amputation mechanisms 

We will show the four different amputation mechanisms: UM, PM, MCAR and MNAR. 

In [ ]:
mechanism_dict = {"mem": "Mutually exclusive missing",
                  "pm": "Partial missing",
                  "mcar": "Missing completely at random",
                  "mnar": "Missing not at random",
                  }

In [ ]:
samples_dict = {}
fig,axs = plt.subplots(1,5, figsize= (12.5,2.5), gridspec_kw={'width_ratios': [4, 4, 4, 4, 0.5]})
for idx, (ax, mechanism) in enumerate(zip(axs, list(mechanism_dict.keys()))):
    transformed_Xs = Amputer(mechanism=mechanism, p=0.8, random_state=RANDOM_STATE).fit_transform(Xs)
    observed_view_indicator = get_observed_mod_indicator(transformed_Xs).sort_values(list(range(len(transformed_Xs))))
    observed_view_indicator.columns = observed_view_indicator.columns + 1
    ax.pcolor(observed_view_indicator, cmap="binary", edgecolors="black", vmin=0., vmax=2.)
    ax.set_title(mechanism_dict[mechanism])
    ax.set_xticks(np.arange(0.5, len(observed_view_indicator.columns), 1), observed_view_indicator.columns)
    ax.set_yticks(np.arange(1.5, len(observed_view_indicator), 2), np.arange(2, len(observed_view_indicator)+2, 2))
    ax.set_xlabel(xlabel), ax.set_ylabel(ylabel)
    if idx != 0:
        ax.get_yaxis().set_visible(False)
    samples_dict[mechanism_dict[mechanism]] = explore.exploring_functions.get_summary(Xs=transformed_Xs)

ax = axs[-1]
ax.pcolor(pd.DataFrame([1, 1, 1, 1, 1, 2, 2, 3, 0, 0]), cmap="gray", edgecolors="black")
ax.set_title("Factor")
ax.axis("off")

plt.tight_layout()

plt.savefig("paper_figures/amputation.pdf")
plt.savefig("paper_figures/amputation.svg")

As shown in the below table, all samples have the same number of complete and incomplete samples. However, the amount of observed samples in each modality varies depending on the chosen missing pattern.

In [ ]:
pd.DataFrame.from_dict(samples_dict, orient= "index")

In [ ]:
print(pd.DataFrame.from_dict(samples_dict, orient= "index").drop(
    columns=["% Observed samples per modality", "Missing samples per modality", "% Missing samples per modality"]).to_latex())

In [ ]:
n_views = 5
n_samples = 100
Xs = [pd.DataFrame(np.random.default_rng(RANDOM_STATE).random((n_samples, 10))) for i in range(n_views)]
for p in np.arange(0.1, 1., 0.1):
    samples_dict = {}
    fig,axs = plt.subplots(1,4, figsize= (12,2.5))
    for idx, (ax, mechanism) in enumerate(zip(axs, list(mechanism_dict.keys()))):
        transformed_Xs = Amputer(mechanism=mechanism, p=p, random_state=RANDOM_STATE+1).fit_transform(Xs)
        observed_view_indicator = get_observed_mod_indicator(transformed_Xs).sort_values(list(range(len(transformed_Xs))))
        observed_view_indicator.columns = observed_view_indicator.columns + 1
        ax.pcolor(observed_view_indicator, cmap="binary", vmin=0., vmax=2.)
        ax.set_title(mechanism_dict[mechanism])
        ax.set_xticks(np.arange(0.5, len(observed_view_indicator.columns), 1), observed_view_indicator.columns)
        ax.set_xlabel(xlabel), ax.set_ylabel(ylabel)
        if idx != 0:
            ax.get_yaxis().set_visible(False)
        samples_dict[mechanism] = explore.exploring_functions.get_summary(Xs=transformed_Xs)
    plt.tight_layout()

    display(pd.DataFrame.from_dict(samples_dict, orient= "index"))